# 서울 지하철 1호선 스케줄 데이터 분석
## 수정 및 개선된 버전

In [1]:
# 필요한 라이브러리 임포트
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import sqlite3
from datetime import datetime, timedelta

# DB 경로 및 테이블 설정
DB_PATH = r"..\data\metro_datetime.db"
TABLE = "Metro_Line_1_Schedule"

## 1. 테이블 속성의 타입 확인
데이터베이스의 각 컬럼이 어떤 데이터 타입으로 저장되어 있는지 확인합니다.

In [2]:
# 전체 데이터 타입 확인
with sqlite3.connect(DB_PATH) as conn:
    query = f"SELECT * FROM {TABLE}"
    df_full = pd.read_sql(query, conn)

# 데이터 타입 표시
print("📊 컬럼별 데이터 타입:")
display(df_full.dtypes.to_frame(name='dtype'))

# 기본 정보 출력
print(f"\n총 데이터 행 수: {len(df_full):,}")
print(f"총 컬럼 수: {len(df_full.columns)}")

📊 컬럼별 데이터 타입:


,dtype
고유번호,int64
호선,int64
역사코드,int64
역사명,object
주중주말,object
방향,object
급행여부,int64
열차코드,object
열차도착시간,object
열차출발시간,object



총 데이터 행 수: 88,431
총 컬럼 수: 14


## 2. 전체 데이터 확인 및 저장
**수정 사항**: `sort_values()` 결과를 올바르게 저장하도록 수정

In [3]:
# 데이터를 시간순으로 정렬하여 저장
# ✅ 수정: sort_values() 결과를 변수에 재할당
df_sorted = df_full.sort_values("열차도착시간_dt")

# CSV로 저장 (한글 깨짐 방지를 위해 utf-8-sig 사용)
df_sorted.to_csv("전체데이터_시간순.csv", index=False, encoding='utf-8-sig')

print("✅ 데이터가 '전체데이터_시간순.csv'로 저장되었습니다.")
print(f"\n처음 5개 행:")
display(df_sorted.head())

✅ 데이터가 '전체데이터_시간순.csv'로 저장되었습니다.

처음 5개 행:


,고유번호,호선,역사코드,역사명,주중주말,방향,급행여부,열차코드,열차도착시간,열차출발시간,출발역,도착역,열차도착시간_dt,열차출발시간_dt
73062,73063,1,1815,부개,DAY,DOWN,1,K1145,0:00:00,18:56:00,용산,동인천,2025-07-04 00:00:00,2025-07-04 18:56:00
74418,74419,1,1816,간석,DAY,UP,1,K1128,0:00:00,18:15:00,동인천,용산,2025-07-04 00:00:00,2025-07-04 18:15:00
74419,74420,1,1816,간석,DAY,UP,1,K1130,0:00:00,18:22:00,동인천,용산,2025-07-04 00:00:00,2025-07-04 18:22:00
49357,49358,1,1715,세류,SAT,UP,1,K1944,0:00:00,19:23:00,천안,청량리,2025-07-04 00:00:00,2025-07-04 19:23:00
49358,49359,1,1715,세류,SAT,UP,1,K1946,0:00:00,20:25:00,천안,청량리,2025-07-04 00:00:00,2025-07-04 20:25:00


## 3. 전체 역의 수 확인
1호선에 포함된 고유한 역의 개수를 확인합니다.

In [4]:
# SQL로 역의 수 확인
with sqlite3.connect(DB_PATH) as conn:
    query = f"""
        SELECT COUNT(DISTINCT `역사코드`) as 역의수
        FROM {TABLE}
    """
    result = pd.read_sql(query, conn)

print(f"🚉 1호선 전체 역의 수: {result['역의수'].iloc[0]}개")

# 추가: 역사명 리스트도 확인
unique_stations = df_full['역사명'].unique()
print(f"\n역사명 목록 (총 {len(unique_stations)}개):")
print(sorted(unique_stations))

🚉 1호선 전체 역의 수: 102개

역사명 목록 (총 102개):
['가능', '가산디지털단지', '간석', '개봉', '관악', '광명', '광운대', '구로', '구일', '군포', '금정', '금천구청', '남영', '노량진', '녹양', '녹천', '당정', '대방', '덕계', '덕정', '도봉', '도봉산', '도원', '도화', '독산', '동대문', '동두천', '동두천중앙', '동묘앞', '동암', '동인천', '두정', '망월사', '명학', '방학', '배방', '백운', '병점', '보산', '봉명', '부개', '부천', '부평', '서동탄', '서울역', '서정리', '석계', '석수', '성균관대', '성환', '세류', '세마', '소사', '소요산', '송내', '송탄', '수원', '시청', '신길', '신도림', '신설동', '신이문', '신창', '쌍용', '아산', '안양', '양주', '역곡', '연천', '영등포', '오류동', '오산', '오산대', '온수', '온양온천', '외대앞', '용산', '월계', '의왕', '의정부', '인천', '전곡', '제기동', '제물포', '종각', '종로3가', '종로5가', '주안', '중동', '지행', '직산', '진위', '창동', '천안', '청량리', '청산', '탕정', '평택', '평택지제', '화서', '회기', '회룡']


## 4. 데이터 품질 검증
분석 전에 데이터의 품질을 확인합니다.

In [5]:
# 4.1 결측값 확인
print("📋 컬럼별 결측값 개수:")
null_counts = df_full.isnull().sum()
display(null_counts[null_counts > 0])

# 4.2 중복 데이터 확인 (같은 열차가 같은 역에 여러 번 기록된 경우)
print("\n🔍 역별 열차별 중복 기록 확인:")
duplicates = df_full.groupby(['열차코드', '역사명', '열차도착시간_dt']).size()
duplicates_found = duplicates[duplicates > 1]
if len(duplicates_found) > 0:
    print(f"중복 발견: {len(duplicates_found)}건")
    display(duplicates_found.head(10))
else:
    print("중복 없음")

# 4.3 주중주말 분포
print("\n📅 주중/주말 데이터 분포:")
display(df_full['주중주말'].value_counts())

📋 컬럼별 결측값 개수:


열차도착시간       2288
열차출발시간       2282
열차도착시간_dt    2288
열차출발시간_dt    2282
dtype: int64


🔍 역별 열차별 중복 기록 확인:
중복 발견: 27840건


열차코드  역사명  열차도착시간_dt          
K1    간석   2025-07-04 05:32:30    3
      개봉   2025-07-04 05:04:00    3
      구일   2025-07-04 05:02:00    3
      도원   2025-07-04 05:40:30    3
      도화   2025-07-04 05:36:30    3
      동암   2025-07-04 05:30:00    3
      동인천  2025-07-04 05:42:30    3
      백운   2025-07-04 05:27:30    3
      부개   2025-07-04 05:22:30    3
      부천   2025-07-04 05:16:00    3
dtype: int64


📅 주중/주말 데이터 분포:


주중주말
DAY    31303
END    28564
SAT    28564
Name: count, dtype: int64

## 5. 기본 통계 분석

In [6]:
# 5.1 운행 열차 수 (열차코드 고유값)
print("🚇 총 운행 열차 수:")
train_counts = df_full.groupby('주중주말')['열차코드'].nunique()
display(train_counts.to_frame(name='열차수'))

# 5.2 방향별 운행 횟수
print("\n⬆️⬇️ 방향별 운행 데이터:")
direction_counts = df_full.groupby(['주중주말', '방향'])['열차코드'].nunique()
display(direction_counts.to_frame(name='열차수'))

# 5.3 급행 vs 일반 열차
print("\n🚄 급행 여부별 데이터:")
express_counts = df_full.groupby(['주중주말', '급행여부'])['열차코드'].nunique()
display(express_counts.to_frame(name='열차수'))

🚇 총 운행 열차 수:


,열차수
주중주말,
DAY,840
END,730
SAT,730



⬆️⬇️ 방향별 운행 데이터:


열차수
주중주말 방향       
DAY  DOWN  420
     UP    420
END  DOWN  368
     UP    362
SAT  DOWN  368
     UP    362


🚄 급행 여부별 데이터:


열차수
주중주말 급행여부     
DAY  0     566
     1     274
END  0     533
     1     197
SAT  0     533
     1     197

## 6. 운행 구간 상세 분석
**수정 사항**: GROUP BY 대신 명확한 쿼리 사용

In [7]:
# 서울역 출발 일반열차 조회 (수정된 쿼리)
with sqlite3.connect(DB_PATH) as conn:
    # ✅ 수정: DISTINCT 사용으로 의도를 명확히 함
    query = f"""
        SELECT DISTINCT
            `역사명`,
            `열차코드`,
            strftime("%H:%M:%S", `열차도착시간_dt`) AS `열차도착시간`,
            strftime("%H:%M:%S", `열차출발시간_dt`) AS `열차출발시간`,
            `출발역`,
            `도착역`,
            `방향`,
            `주중주말`
        FROM {TABLE}
        WHERE `급행여부` = 0 AND `출발역` = "서울역" AND `주중주말` = "DAY"
        ORDER BY `열차코드`, `열차도착시간_dt`
    """
    df_seoul_routes = pd.read_sql(query, conn)

# CSV로 저장
df_seoul_routes.to_csv('서울역출발_운행구간.csv', index=False, encoding='utf-8-sig')

print(f"✅ 서울역 출발 일반열차: 총 {len(df_seoul_routes)}개 정차 기록")
print(f"고유 열차 수: {df_seoul_routes['열차코드'].nunique()}대\n")
print("처음 50개 행:")
display(df_seoul_routes.head(50))

✅ 서울역 출발 일반열차: 총 177개 정차 기록
고유 열차 수: 8대

처음 50개 행:


,역사명,열차코드,열차도착시간,열차출발시간,출발역,도착역,방향,주중주말
0,서울역,S401,None,05:20:00,서울역,서동탄,DOWN,DAY
1,남영,S401,05:22:30,05:23:00,서울역,서동탄,DOWN,DAY
2,용산,S401,05:25:00,05:25:30,서울역,서동탄,DOWN,DAY
3,노량진,S401,05:28:30,05:29:00,서울역,서동탄,DOWN,DAY
4,대방,S401,05:31:00,05:31:30,서울역,서동탄,DOWN,DAY
5,신길,S401,05:32:30,05:33:00,서울역,서동탄,DOWN,DAY
6,영등포,S401,05:35:00,05:35:30,서울역,서동탄,DOWN,DAY
7,신도림,S401,05:37:30,05:38:00,서울역,서동탄,DOWN,DAY
8,구로,S401,05:40:00,05:40:30,서울역,서동탄,DOWN,DAY
9,가산디지털단지,S401,05:44:30,05:45:00,서울역,서동탄,DOWN,DAY


## 7. 운행 구간별 통계

In [8]:
# 7.1 출발역-도착역 조합별 열차 수
print("🔀 운행 구간별 열차 수:")
route_summary = df_full.groupby(['출발역', '도착역', '주중주말']).agg({
    '열차코드': 'nunique',
    '역사명': 'count'
}).rename(columns={'열차코드': '열차수', '역사명': '정차기록수'})

display(route_summary.sort_values('열차수', ascending=False).head(20))

# 7.2 주요 운행 구간 시각화
route_day = route_summary.loc[(slice(None), slice(None), 'DAY'), :].reset_index()
route_day['구간'] = route_day['출발역'] + ' → ' + route_day['도착역']

fig = px.bar(
    route_day.nlargest(15, '열차수'),
    x='구간',
    y='열차수',
    title='주중 주요 운행 구간별 열차 수 (Top 15)',
    labels={'구간': '운행 구간', '열차수': '열차 대수'},
    color='열차수',
    color_continuous_scale='Blues'
)
fig.update_layout(xaxis_tickangle=-45)
fig.show()

🔀 운행 구간별 열차 수:


열차수  정차기록수
출발역 도착역 주중주말            
동인천 용산  DAY    78   2028
용산  동인천 DAY    75   1950
동인천 용산  END    72   1872
        SAT    72   1872
용산  동인천 SAT    69   1725
        END    69   1725
광운대 서동탄 END    34   1360
        SAT    34   1360
서동탄 광운대 END    32   1280
        SAT    32   1280
광운대 서동탄 DAY    32   1280
서동탄 광운대 DAY    30   1200
인천  양주  DAY    30   1650
        END    28   1540
        SAT    28   1540
양주  인천  DAY    27   1485
천안  청량리 SAT    26   1208
        END    26   1208
양주  인천  END    26   1430
        SAT    26   1430

## 8. 특정 열차의 전체 여정 확인

In [9]:
def get_train_journey(df, train_code, day_type='DAY'):
    """
    특정 열차코드의 전체 여정을 시간순으로 반환
    
    Parameters:
    -----------
    df : DataFrame
        전체 데이터
    train_code : str
        열차 코드 (예: 'S401')
    day_type : str
        'DAY' 또는 'SAT'
    
    Returns:
    --------
    DataFrame
        해당 열차의 전체 정차역 정보
    """
    journey = df[
        (df['열차코드'] == train_code) & 
        (df['주중주말'] == day_type)
    ].sort_values('열차도착시간_dt')
    
    return journey[[
        '역사명', '열차도착시간', '열차출발시간', 
        '출발역', '도착역', '방향'
    ]]

# 예시: S401 열차의 여정
train_code_example = 'S401'
journey = get_train_journey(df_full, train_code_example)

print(f"🚆 열차 {train_code_example}의 전체 여정:")
print(f"출발역: {journey.iloc[0]['출발역']}")
print(f"도착역: {journey.iloc[-1]['도착역']}")
print(f"방향: {journey.iloc[0]['방향']}")
print(f"총 정차역: {len(journey)}개\n")
display(journey)

🚆 열차 S401의 전체 여정:
출발역: 서울역
도착역: 서동탄
방향: DOWN
총 정차역: 26개



,역사명,열차도착시간,열차출발시간,출발역,도착역,방향
14068,남영,5:22:30,5:23:00,서울역,서동탄,DOWN
15460,용산,5:25:00,5:25:30,서울역,서동탄,DOWN
17332,노량진,5:28:30,5:29:00,서울역,서동탄,DOWN
19204,대방,5:31:00,5:31:30,서울역,서동탄,DOWN
32734,신길,5:32:30,5:33:00,서울역,서동탄,DOWN
21098,영등포,5:35:00,5:35:30,서울역,서동탄,DOWN
23094,신도림,5:37:30,5:38:00,서울역,서동탄,DOWN
36229,구로,5:40:00,5:40:30,서울역,서동탄,DOWN
38263,가산디지털단지,5:44:30,5:45:00,서울역,서동탄,DOWN
47803,독산,5:47:00,5:47:30,서울역,서동탄,DOWN


## 9. 시간대별 운행 패턴 분석

In [10]:
# 출발 시간대별 열차 수 집계
# datetime 변환 (24시간 초과 처리)
def parse_time_with_overflow(time_str):
    """24시간을 초과하는 시간 문자열을 처리"""
    if pd.isna(time_str) or time_str is None:
        return None
    
    try:
        # 이미 datetime 객체인 경우
        if isinstance(time_str, datetime):
            return time_str
        
        # 문자열인 경우 파싱
        time_str = str(time_str).strip()
        
        # HH:MM:SS 형식인 경우
        if ':' in time_str:
            parts = time_str.split(':')
            hour = int(parts[0])
            minute = int(parts[1])
            second = int(parts[2]) if len(parts) > 2 else 0
            
            # 24시간 이상인 경우 modulo 연산
            if hour >= 24:
                hour = hour % 24
            
            return datetime(2024, 1, 1, hour, minute, second)
        
        # 그 외의 경우 pandas로 파싱 시도
        return pd.to_datetime(time_str)
    except:
        return None

# 시간 파싱 적용
df_time = df_full[df_full['열차출발시간'].notna()].copy()
df_time['출발시간_parsed'] = df_time['열차출발시간'].apply(parse_time_with_overflow)
df_time['시간대'] = df_time['출발시간_parsed'].dt.hour

# 시간대별 출발 열차 수
hourly_departures = df_time.groupby(['주중주말', '시간대', '출발역'])['열차코드'].nunique().reset_index()
hourly_departures.rename(columns={'열차코드': '출발열차수'}, inplace=True)

# 서울역 기준 시각화
seoul_hourly = hourly_departures[hourly_departures['출발역'] == '서울역']

fig = px.line(
    seoul_hourly,
    x='시간대',
    y='출발열차수',
    color='주중주말',
    title='서울역 시간대별 출발 열차 수',
    labels={'시간대': '시간 (시)', '출발열차수': '출발 열차 대수'},
    markers=True
)
fig.add_vrect(x0=7, x1=9, fillcolor="red", opacity=0.1, annotation_text="출근 시간")
fig.add_vrect(x0=18, x1=20, fillcolor="blue", opacity=0.1, annotation_text="퇴근 시간")
fig.show()

print("\n📊 서울역 시간대별 출발 열차 통계:")
display(seoul_hourly.pivot(index='시간대', columns='주중주말', values='출발열차수').fillna(0))


📊 서울역 시간대별 출발 열차 통계:


주중주말,DAY,END,SAT
시간대,,,
0,1.0,0.0,0.0
5,5.0,4.0,4.0
6,7.0,4.0,4.0
7,2.0,0.0,0.0


## 10. 배차 간격 분석 (Headway)
특정 역에서 열차가 몇 분 간격으로 도착하는지 분석합니다.

In [11]:
def calculate_headway(df, station_name, direction, day_type='DAY'):
    """
    특정 역의 특정 방향 배차 간격 계산
    
    Parameters:
    -----------
    df : DataFrame
        전체 데이터
    station_name : str
        역 이름 (예: '서울역')
    direction : str
        'UP' 또는 'DOWN'
    day_type : str
        'DAY' 또는 'SAT'
    
    Returns:
    --------
    DataFrame
        배차 간격 정보
    """
    # 필터링
    station_df = df[
        (df['역사명'] == station_name) & 
        (df['방향'] == direction) & 
        (df['주중주말'] == day_type) &
        (df['열차도착시간_dt'].notna())
    ].copy()
    
    # 시간순 정렬
    station_df = station_df.sort_values('열차도착시간_dt')
    
    # datetime 변환
    station_df['도착시간_parsed'] = pd.to_datetime(station_df['열차도착시간_dt'])
    
    # 배차 간격 계산 (분 단위)
    station_df['배차간격_분'] = station_df['도착시간_parsed'].diff().dt.total_seconds() / 60
    
    return station_df[[
        '열차코드', '열차도착시간', '배차간격_분', 
        '출발역', '도착역'
    ]]

# 예시: 서울역 하행 배차 간격
headway_seoul_down = calculate_headway(df_full, '서울역', 'DOWN', 'DAY')

print("⏱️ 서울역 하행(DOWN) 배차 간격 분석 (주중):")
print(f"\n평균 배차 간격: {headway_seoul_down['배차간격_분'].mean():.1f}분")
print(f"최소 배차 간격: {headway_seoul_down['배차간격_분'].min():.1f}분")
print(f"최대 배차 간격: {headway_seoul_down['배차간격_분'].max():.1f}분")

display(headway_seoul_down.head(20))

# 배차 간격 분포 시각화
fig = px.histogram(
    headway_seoul_down,
    x='배차간격_분',
    nbins=30,
    title='서울역 하행 배차 간격 분포 (주중)',
    labels={'배차간격_분': '배차 간격 (분)', 'count': '빈도'},
    color_discrete_sequence=['steelblue']
)
fig.show()

⏱️ 서울역 하행(DOWN) 배차 간격 분석 (주중):

평균 배차 간격: 4.9분
최소 배차 간격: 3.0분
최대 배차 간격: 30.5분


,열차코드,열차도착시간,배차간격_분,출발역,도착역
3,K11,5:30:00,NaN,광운대,인천
11,K13,5:37:00,7.0,창동,인천
60,K405,5:42:00,5.0,광운대,서동탄
173,S15,5:48:30,6.5,동묘앞,인천
184,S407,5:53:30,5.0,동묘앞,서동탄
26,K17,5:59:00,5.5,의정부,인천
36,K19,6:05:00,6.0,양주,인천
61,K409,6:10:00,5.0,광운대,서동탄
47,K21,6:14:00,4.0,양주,인천
216,K1901,6:18:30,4.5,청량리,신창


## 11. 역별 정차 열차 수 분석

In [12]:
# 역별 정차 열차 수 (주중 기준)
station_traffic = df_full[df_full['주중주말'] == 'DAY'].groupby('역사명').agg({
    '열차코드': 'nunique',
    '고유번호': 'count'
}).rename(columns={
    '열차코드': '정차열차수',
    '고유번호': '총정차횟수'
}).sort_values('정차열차수', ascending=False)

print("🚉 역별 정차 열차 수 Top 20 (주중):")
display(station_traffic.head(20))

# 시각화
fig = px.bar(
    station_traffic.head(20).reset_index(),
    x='역사명',
    y='정차열차수',
    title='역별 정차 열차 수 Top 20 (주중)',
    labels={'역사명': '역 이름', '정차열차수': '정차 열차 대수'},
    color='정차열차수',
    color_continuous_scale='Viridis'
)
fig.update_layout(xaxis_tickangle=-45)
fig.show()

🚉 역별 정차 열차 수 Top 20 (주중):


,정차열차수,총정차횟수
역사명,,
구로,777,777
영등포,696,696
신도림,696,696
신길,654,654
대방,654,654
노량진,654,654
용산,654,654
서울역,483,483
시청,482,482


## 12. 주중/주말 비교 분석

In [13]:
# 주중/주말 비교 통계
comparison = df_full.groupby('주중주말').agg({
    '열차코드': 'nunique',
    '역사명': lambda x: x.nunique(),
    '고유번호': 'count'
}).rename(columns={
    '열차코드': '총_열차수',
    '역사명': '운행_역수',
    '고유번호': '총_정차횟수'
})

print("📅 주중/주말 운행 비교:")
display(comparison)

# 차이 계산
if 'DAY' in comparison.index and 'SAT' in comparison.index:
    diff = comparison.loc['DAY'] - comparison.loc['SAT']
    print("\n📊 주중-주말 차이:")
    display(diff.to_frame(name='차이'))

# 시각화
fig = go.Figure(data=[
    go.Bar(name='주중', x=comparison.columns, y=comparison.loc['DAY'] if 'DAY' in comparison.index else []),
    go.Bar(name='주말', x=comparison.columns, y=comparison.loc['SAT'] if 'SAT' in comparison.index else [])
])
fig.update_layout(
    title='주중/주말 운행 비교',
    xaxis_title='지표',
    yaxis_title='값',
    barmode='group'
)
fig.show()

📅 주중/주말 운행 비교:


,총_열차수,운행_역수,총_정차횟수
주중주말,,,
DAY,840,102,31303
END,730,102,28564
SAT,730,102,28564



📊 주중-주말 차이:


,차이
총_열차수,110
운행_역수,0
총_정차횟수,2739


## 💡 분석 결론 및 다음 단계

### 주요 발견사항:
1. 데이터 구조 및 품질 확인 완료
2. 운행 구간 및 시간대별 패턴 파악
3. 배차 간격 분석을 통한 운행 빈도 확인

### 추가 분석 방향:
1. **혼잡도 분석**: 시간대별 배차 간격과 실제 혼잡도 연관성 파악
2. **차량 노후도 연동**: 차량 노후도 데이터와 운행 패턴 결합
3. **구간별 소요시간**: 역 간 이동 시간 분석
4. **지연 분석**: 계획 vs 실제 운행 시간 비교 (실시간 데이터 필요)
5. **시각화 고도화**: 간트 차트, 히트맵 등으로 시공간 패턴 표현

### 데이터 통합:
- 차량 노후도 데이터
- 실시간 혼잡도 API 데이터
- 승하차 인원 통계
를 결합하여 종합적인 혼잡도 원인 분석 가능